In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
storm_sewers = gpd.read_file('Storm_Sewer_Pipes.geojson')

In [ ]:
storm_sewers.head()

In [ ]:
storm_sewers.info(verbose=True)

In [ ]:
storm_sewers_narrow = storm_sewers[['OBJECTID','Dim_1','Legacy_pipe_length','Existing_size', 'Material','Owner','District_Code', 'Label','ShapeSTLength','geometry']]
storm_sewers_narrow.head(50)

In [ ]:
storm_sewers_narrow.plot()


In [ ]:
storm_sewers_narrow.explore()

In [ ]:
alders = gpd.read_file('alder_districts_without_lakes.geojson')

In [ ]:
alders.head()

In [ ]:
join_left_storm_sewer_alder = storm_sewers_narrow.sjoin(alders, how="left")

join_left_storm_sewer_alder


In [ ]:
sewer_length_alder = join_left_storm_sewer_alder[['ShapeSTLength','ALD_DIST']].groupby(['ALD_DIST']).sum()
sewer_length_alder=sewer_length_alder.reset_index(level=['ALD_DIST'])
sewer_length_alder['ALD_DIST']=sewer_length_alder.ALD_DIST.astype(int)
sewer_length_alder= sewer_length_alder.rename(columns={"ShapeSTLength": "storm_sewer_length"})
sewer_length_alder.set_index('ALD_DIST')
sewer_length_alder.head(50)

In [ ]:
alders_storm_sewers = pd.merge(alders,sewer_length_alder, left_on='ALD_DIST', right_on='ALD_DIST', how='left')
alders_storm_sewers.head()

In [ ]:
alder_data= pd.read_pickle("alder_data.pkl")
alder_data.head(50)

In [ ]:
alders_storm_sewers_taxes = pd.merge(alder_data,alders_storm_sewers, left_on='AlderDistrict', right_on='ALD_DIST', how='right')
alders_storm_sewers_taxes  = gpd.GeoDataFrame(alders_storm_sewers_taxes , geometry='geometry')
alders_storm_sewers_taxes.head(30)

In [ ]:
alders_storm_sewers_taxes['storms_sewers_per_dwelling_unit'] = alders_storm_sewers_taxes['storm_sewer_length']/alders_storm_sewers_taxes['TotalDwellingUnits']
alders_storm_sewers_taxes['taxes_per_foot_storms_sewers'] = alders_storm_sewers_taxes['NetTaxes']/alders_storm_sewers_taxes['storm_sewer_length']

alders_storm_sewers_taxes.head(25)


In [ ]:
alders_storm_sewers_taxes.plot(column='storms_sewers_per_dwelling_unit', cmap='OrRd', edgecolor='k', legend=True)

In [ ]:
alders_storm_sewers_taxes.plot(column='taxes_per_foot_storms_sewers', cmap='OrRd', edgecolor='k', legend=True)

In [ ]:
x_map=alders_storm_sewers_taxes.centroid.x.mean()
y_map=alders_storm_sewers_taxes.centroid.y.mean()

In [ ]:
dif_map = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(dif_map)

folium.Choropleth(
    geo_data=alders_storm_sewers_taxes,
    data=alders_storm_sewers_taxes,
    columns=['ALD_DIST','storms_sewers_per_dwelling_unit'],
    key_on="feature.properties.ALD_DIST",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Feet of storm sewers per dwelling unit",
    highlight=True,
).add_to(dif_map)

dif_map

In [ ]:
taxes_storm_sewers_map = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(taxes_storm_sewers_map)

folium.Choropleth(
    geo_data=alders_storm_sewers_taxes,
    data=alders_storm_sewers_taxes,
    columns=['ALD_DIST','taxes_per_foot_storms_sewers'],
    key_on="feature.properties.ALD_DIST",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Dollars of taxes per foot of storm sewer",
    highlight=True,
).add_to(taxes_storm_sewers_map)

taxes_storm_sewers_map

In [ ]:
alders_storm_sewers_taxes.explore("taxes_per_foot_storms_sewers", cmap="Blues") 

In [ ]:
nb_assoc = gpd.read_file('Neighborhood_Associations.geojson')
nb_assoc.explore()

In [ ]:
bay_creek = nb_assoc[nb_assoc['NEIGHB_NAME']== 'Bay Creek Neighborhood Association']
bay_creek.explore()

In [ ]:
bay_creek.head()

In [ ]:
sewers_int_bay_creek = bay_creek.overlay(storm_sewers_narrow, how='intersection', keep_geom_type=False)
sewers_int_bay_creek.explore()

In [ ]:
storm_sewers_int_bay_creek_grouped = sewers_int_bay_creek[['NA_ID', 'Legacy_pipe_length','ShapeSTLength_2']].groupby(['NA_ID']).sum()
storm_sewers_int_bay_creek_grouped

In [ ]:
sanitary_sewers = gpd.read_file('Sanitary_Sewer_Mains.geojson')
sanitary_sewers.info(verbose=True)

In [ ]:
sanitary_sewers = sanitary_sewers[['OBJECTID','Length_ft','ShapeSTLength','geometry']]

sanitary_sewers.explore()

In [ ]:
sanitary_sewers_int_bay_creek = bay_creek.overlay(sanitary_sewers, how='intersection', keep_geom_type=False)
sanitary_sewers_int_bay_creek = sanitary_sewers_int_bay_creek[['NA_ID','OBJECTID_2','Length_ft','ShapeSTLength_2','geometry']]
sanitary_sewers_int_bay_creek.explore()

In [ ]:
sanitary_sewers_int_bay_creek_grouped = sanitary_sewers_int_bay_creek[['NA_ID', 'Length_ft','ShapeSTLength_2']].groupby(['NA_ID']).sum()
sanitary_sewers_int_bay_creek_grouped

In [ ]:
sanitary_sewer_laterals = gpd.read_file('Sanitary_Sewer_Laterals.geojson')
sanitary_sewer_laterals.info(verbose=True)

In [ ]:
sanitary_sewer_laterals = sanitary_sewer_laterals[['OBJECTID','Parcel_no','SHAPESTLength','geometry']]

sanitary_sewer_laterals.explore()

In [ ]:
sanitary_sewer_laterals_int_bay_creek = bay_creek.overlay(sanitary_sewer_laterals, how='intersection', keep_geom_type=False)
#sanitary_sewer_laterals_int_bay_creek = sanitary_sewers_laterals_int_bay_creek[['NA_ID','OBJECTID','Parcel_no','SHAPESTLength','geometry']]
sanitary_sewer_laterals_int_bay_creek.explore()

In [ ]:
sanitary_sewer_laterals_bay_creek_grouped = sanitary_sewer_laterals_int_bay_creek[['NA_ID', 'SHAPESTLength']].groupby(['NA_ID']).sum()
sanitary_sewer_laterals_bay_creek_grouped

In [ ]:
streets =  gpd.read_file('Street_Centerlines_and_Pavement_Data(1).geojson')
streets.info(verbose=True)

In [ ]:
streets_geomcollection = streets[streets.geom_type == 'GeometryCollection']
streets_geomcollection.head()

In [ ]:
#drop the geomcollections since they are zerolength segments making everything hard
streets = streets[streets.geom_type == 'LineString']
streets.plot()

In [ ]:
 #df2 = df.loc[:, ['A']]

streets_narrow = streets.loc[:,['OBJECTID','segment_name','surface_width','sidewalk','median_width','rw_width','lanes','ShapeSTLength','geometry']]
streets_narrow.explore()

In [ ]:
streets_narrow.head(50)

In [ ]:

streets_narrow['pavement_area'] = (streets_narrow['surface_width'] * streets_narrow['ShapeSTLength'])


streets_narrow['right_of_way_area'] = (streets_narrow['rw_width'] * streets_narrow['ShapeSTLength'])



In [ ]:
streets_int_bay_creek = bay_creek.overlay(streets_narrow, how='intersection', keep_geom_type=False)


In [ ]:
streets_int_bay_creek.explore()


In [ ]:
streets_bay_creek_grouped = streets_int_bay_creek[['NA_ID', 'ShapeSTArea','ShapeSTLength_2','pavement_area','right_of_way_area']].groupby(['NA_ID','ShapeSTArea' ]).sum()
streets_bay_creek_grouped=streets_bay_creek_grouped.reset_index('ShapeSTArea')
streets_bay_creek_grouped['percent_street'] = 100*(streets_bay_creek_grouped['pavement_area']/streets_bay_creek_grouped['ShapeSTArea'])
streets_bay_creek_grouped['percent_row'] = 100*(streets_bay_creek_grouped['right_of_way_area']/streets_bay_creek_grouped['ShapeSTArea'])

streets_bay_creek_grouped.head()

area per neighborhood

In [ ]:
streets_narrow.explore()

In [ ]:
streets_int_neb_assoc = nb_assoc.overlay(streets_narrow, how='intersection', keep_geom_type=False)

In [ ]:
streets_int_neb_assoc.explore()

In [ ]:
streets_neb_assoc_grouped = streets_int_neb_assoc[['NA_ID', 'ShapeSTArea','ShapeSTLength_2','pavement_area','right_of_way_area']].groupby(['NA_ID','ShapeSTArea' ]).sum()
streets_neb_assoc_grouped=streets_neb_assoc_grouped.reset_index('ShapeSTArea')
streets_neb_assoc_grouped['percent_street'] = 100*(streets_neb_assoc_grouped['pavement_area']/streets_neb_assoc_grouped['ShapeSTArea'])
streets_neb_assoc_grouped['percent_row'] = 100*(streets_neb_assoc_grouped['right_of_way_area']/streets_neb_assoc_grouped['ShapeSTArea'])

streets_neb_assoc_grouped.head(10)

In [ ]:
nb_assoc_streets_row_area = pd.merge(nb_assoc,streets_neb_assoc_grouped, left_on='NA_ID', right_on='NA_ID', how='left')
nb_assoc_streets_row_area.head()

In [ ]:
nb_assoc_streets_row_area.explore()

In [ ]:
street_row_map = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(street_row_map)

folium.Choropleth(
    geo_data=nb_assoc_streets_row_area,
    data=nb_assoc_streets_row_area,
    columns=['NEIGHB_NAME','percent_street'],
    key_on="feature.properties.NEIGHB_NAME",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Percentage of Neighborhood that is paved street",
    highlight=True,
).add_to(street_row_map)

street_row_map

In [ ]:
neighb_assoc_tax_data = pd.read_csv('neigh_associan_value_taxes.csv')
neighb_assoc_tax_data.head()

In [ ]:
nb_assoc_streets_row_area_taxes = pd.merge(nb_assoc_streets_row_area,neighb_assoc_tax_data, left_on='NA_ID', right_on='NA_ID', how='left')
nb_assoc_streets_row_area_taxes['taxes_per_sq_foot_street'] = nb_assoc_streets_row_area_taxes['NetTaxes']/nb_assoc_streets_row_area_taxes['pavement_area']
nb_assoc_streets_row_area_taxes['sq_foot_street_per_dwelling_unit'] = nb_assoc_streets_row_area_taxes['pavement_area']/nb_assoc_streets_row_area_taxes['TotalDwellingUnits']

nb_assoc_streets_row_area_taxes['taxes_per_sq_foot_street']= nb_assoc_streets_row_area_taxes['taxes_per_sq_foot_street'].fillna(0)
#nb_assoc_streets_row_area_taxes['sq_foot_street_per_dwelling_unit']= nb_assoc_streets_row_area_taxes['sq_foot_street_per_dwelling_unit'].fillna(0)

nb_assoc_streets_row_area_taxes.replace([np.inf, -np.inf], np.nan, inplace=True)

nb_assoc_streets_row_area_taxes.to_csv('taxes_per_sq_foot_street.csv')
nb_assoc_streets_row_area_taxes.info()

In [ ]:
taxes_per_street_map = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(taxes_per_street_map)

cp= folium.Choropleth(
    geo_data=nb_assoc_streets_row_area_taxes,
    data=nb_assoc_streets_row_area_taxes,
    columns=['NEIGHB_NAME_x','taxes_per_sq_foot_street'],
    key_on="feature.properties.NEIGHB_NAME_x",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Net Taxes per square foot of street",
    highlight=True,
).add_to(taxes_per_street_map)

folium.GeoJsonTooltip(['NEIGHB_NAME_x','taxes_per_sq_foot_street']).add_to(cp.geojson)
  
#folium.LayerControl().add_to(m)

taxes_per_street_map

In [ ]:
url = (
      "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
  )
  state_geo = f"{url}/us-states.json"
  state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
  state_data = pd.read_csv(state_unemployment)
  
  m = folium.Map(location=[48, -102], zoom_start=3)
  
  # capturing the return of folium.Choropleth()
  cp = folium.Choropleth(
      geo_data=state_geo,
      name="choropleth",
      data=state_data,
      columns=["State", "Unemployment"],
      key_on="feature.id",
      fill_color="YlGn",
      fill_opacity=0.7,
      line_opacity=0.2,
      legend_name="Unemployment Rate (%)",
  ).add_to(m)
  
  # creating a state indexed version of the dataframe so we can lookup values
  state_data_indexed = state_data.set_index('State')
  
  # looping thru the geojson object and adding a new property(unemployment)
  # and assigning a value from our dataframe
  for s in cp.geojson.data['features']:
      s['properties']['unemployment'] = state_data_indexed.loc[s['id'], 'Unemployment']
  
  # and finally adding a tooltip/hover to the choropleth's geojson
  folium.GeoJsonTooltip(['name', 'unemployment']).add_to(cp.geojson)
  
  folium.LayerControl().add_to(m)
  
  m

In [ ]:
pavement_per_dwelling_unit_map = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(pavement_per_dwelling_unit_map)

folium.Choropleth(
    geo_data=nb_assoc_streets_row_area_taxes,
    data=nb_assoc_streets_row_area_taxes,
    columns=['NEIGHB_NAME_x','sq_foot_street_per_dwelling_unit'],
    key_on="feature.properties.NEIGHB_NAME_x",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Square foot of street pavement per dwelling unit",
    highlight=True,
).add_to(pavement_per_dwelling_unit_map)

pavement_per_dwelling_unit_map